<a href="https://colab.research.google.com/github/CameronKenworthyCode/NEWICC/blob/main/2022_fires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#initialize notebook
import pandas as pd
import numpy as np
from datetime import datetime as dt
import re

!pip install pykml
from pykml.factory import KML_ElementMaker as KML

!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

drive.mount('/content/drive/', force_remount=True)

file_list = gdrive.ListFile(
    {'q': "'1fZ-p58kmyIkYnopyDITA0B8tW0me8FXh' in parents and trashed=False"}).GetList() 

In [ ]:
for file in file_list:
  print(file['title'])

In [ ]:
dump_incident_ic = pd.read_csv('drive/My Drive/NEWICC/Dump_IncidentIC.csv', encoding='cp1252')
dump_resource = pd.read_csv('drive/My Drive/NEWICC/Dump_Resource.csv', encoding='cp1252')
dump_incident = pd.read_csv('drive/My Drive/NEWICC/Dump_Incident.csv', encoding='cp1252')
dump_incident_center_fires = pd.read_csv('drive/My Drive/NEWICC/Dump_IncidentCenterFires.csv', encoding='cp1252')
dump_fireinfo = pd.read_csv('drive/My Drive/NEWICC/Dump_FireInfo.csv', encoding='cp1252')
dump_incident_log = pd.read_csv('drive/My Drive/NEWICC/Dump_IncidentLog.csv', encoding='cp1252')
dump_spec_cause = pd.read_csv('drive/My Drive/NEWICC/Dump_SpecCause.csv', encoding='cp1252')
dump_stat_cause = pd.read_csv('drive/My Drive/NEWICC/Dump_StatCause.csv', encoding='cp1252')
dump_incident_type = pd.read_csv('drive/My Drive/NEWICC/Dump_IncidentType.csv', encoding='cp1252')
dump_incident_subtype = pd.read_csv('drive/My Drive/NEWICC/Dump_IncidentSubTypes.csv', encoding='cp1252')
dump_centerfire_subcats = pd.read_csv('drive/My Drive/NEWICC/Dump_CenterFireSubCats.csv', encoding='cp1252')
dump_status_history =  pd.read_csv('drive/My Drive/NEWICC/Dump_StatusHistory.csv', encoding='cp1252')
dump_status =  pd.read_csv('drive/My Drive/NEWICC/Dump_Status.csv', encoding='cp1252')
dump_centerfire_cats = pd.read_csv('drive/My Drive/NEWICC/Dump_CenterFireCats.csv', encoding='cp1252')

In [ ]:
KCT = {
    0: {'type':'N/A', 'subtype':'N/A'},
    1: {'type':'Airtanker', 'subtype':'Type 1'},
    2: {'type':'Airtanker', 'subtype':'Type 2'},
    3: {'type':'Airtanker', 'subtype':'Type 3/4'},
    4: {'type':'Fixed Wing', 'subtype':'Air Attack Platform'},
    5: {'type':'Fixed Wing', 'subtype':'Lead Plane'},
    6: {'type':'Fixed Wing', 'subtype':'Recon Plane'},
    7: {'type':'Fixed Wing', 'subtype':'Smokejumper'},
    8: {'type': 'Helicopter', 'subtype': 'Type 1'},
    9: {'type': 'Helicopter', 'subtype': 'Type 2'},
    10: {'type': 'Helicopter', 'subtype': 'Type 3'},
    11: {'type': 'Handcrew', 'subtype': 'Type 1'},
    12: {'type': 'Handcrew', 'subtype': 'Type 2'},
    13: {'type': 'Handcrew', 'subtype': 'Type 2/IA'},
    14: {'type': 'Handcrew', 'subtype': 'Other'},
    15: {'type': 'Module', 'subtype': 'Helitack'},
    16: {'type': 'Overhead', 'subtype': 'Firefighter'},
    17: {'type': 'Overhead', 'subtype': 'Smokejumper'},
    18: {'type': 'Module', 'subtype': 'Rappeller'},
    19: {'type': 'Dozer', 'subtype': 'Type 1'},
    20: {'type': 'Dozer', 'subtype': 'Type 2'},
    21: {'type': 'Dozer', 'subtype': 'Type 3'},
    25: {'type': 'Engine', 'subtype': 'Other'},
    26: {'type': 'Boat', 'subtype': 'Boat'},
    27: {'type': 'Utility Vehicle', 'subtype': 'High Mobility'},
    28: {'type': 'Utility Vehicle', 'subtype': 'Low Ground Pressure'},
    29: {'type': 'Tractor Plow', 'subtype': 'Trencher'},
    30: {'type': 'Tractor Plow', 'subtype': 'Type 1'},
    31: {'type': 'Tractor Plow', 'subtype': 'Type 2/3'},
    32: {'type': 'Tractor Plow', 'subtype': 'Type 4/5/6'},
    33: {'type': 'Watertender', 'subtype': 'Type 1'},
    34: {'type': 'Watertender', 'subtype': 'Type 2'},
    35: {'type': 'Watertender', 'subtype': 'Type 3'},
    36: {'type': 'Overhead', 'subtype': 'Operations Overhead'},
    51: {'type': 'Engine', 'subtype': 'Type 1'},
    52: {'type': 'Engine', 'subtype': 'Type 2'},
    53: {'type': 'Engine', 'subtype': 'Type 3'},
    54: {'type': 'Engine', 'subtype': 'Type 4'},
    55: {'type': 'Engine', 'subtype': 'Type 5'},
    56: {'type': 'Engine', 'subtype': 'Type 6'},
    57: {'type': 'Engine', 'subtype': 'Type 7'},
    58: {'type': 'Fixed Wing', 'subtype': 'Aerial Supervision'},
    81: {'type': 'Module', 'subtype': 'Aerial Supervision'},
    82: {'type': 'Module', 'subtype': 'Fuels'},
    83: {'type': 'Module', 'subtype': 'Overhead Squad'},
    84: {'type': 'Module', 'subtype': 'Suppression'},
    86: {'type': 'Module', 'subtype': 'Wildland Fire Type 1'},
    87: {'type': 'Module', 'subtype': 'Wildland Fire Type 2'},
    }

def get_incident_df(ID):
  part = dump_incident_log.query('IncidentID == @ID')
  incident = dump_incident.query("IncidentID == @ID")
  incidentnum = incident['IncidentNum'].values
  incidentname = incident['IncidentName'].values
  title_str = incidentname[0] + ' | ' + str(incidentnum[0])
  log = {"LogDate": part['LogDate'].reset_index(drop=True),
        "LogFrom": part['LogFrom'].reset_index(drop=True),
        "LogTo": part['LogTo'].reset_index(drop=True),
        "Entry": part['Describe'].reset_index(drop=True)}
  return pd.DataFrame(log).sort_values(by='LogDate').reset_index(drop=True)

def statID(ID):
  return dump_stat_cause.query("StatCauseID == @ID")['Describe'].values[0]

def specID(ID):
  return dump_spec_cause.query("SpecCauseID == @ID")['Describe'].values[0]

def resourceID(ID):
  return dump_resource.query("ResourceID == @ID")["ResourceCode"].values[0]

def resourceType(ID, only=None):
  if only == 'type':
    return KCT[dump_resource.query("ResourceID == @ID")['KCTID'].values[0]]['type']
  
  else:
    return KCT[dump_resource.query("ResourceID == @ID")['KCTID'].values[0]]

def statusID(ID, only=None):
  status =  dump_status.query("StatusID == @ID")
  describe = status["Describe"].values[0]
  available = status['Available'].values[0]
  on_incident = status["OnIncident"].values[0]
  if only == 'on_incident':
    return on_incident
  else:
    return {"status": describe, "available": available, "on_incident": on_incident}

def incidentType(ID):
  return dump_incident_type.query("IncidentTypeID == @ID")['Describe'].values[0]

def centerfireID(tab, catID):
  return dump_centerfire_cats.query("CenterFireTabID == @tab and CenterFireCatID == @catID")['Describe'].values[0]

def count_response(ID):
  incident = dump_status_history.query("IncidentID == @ID").sort_values(by="StatusDate")
  incident = incident.drop(columns=['StatusDate', 'DispatcherID', 'Describe', 'StatusHistoryID']).drop_duplicates()

  incident['ResourceType'] = pd.DataFrame(incident['ResourceID'].apply(resourceType, only='type'))
  incident = incident.query('StatusID == 3')

  try: 
    response_dict = dict(incident['ResourceType'].value_counts().drop(labels='N/A'))
  except:
    response_dict = dict(incident['ResourceType'].value_counts())

  response_columns = {'Airtanker': 0, 'Fixed Wing': 0, 'Helicopter': 0, 'Handcrew': 0,
       'Module': 0, 'Overhead': 0, 'Dozer': 0, 'Engine': 0, 'Boat': 0, 'Utility Vehicle': 0,
       'Tractor Plow':0, 'Watertender':0}
  
  for key in response_dict:
    response_columns[key] = response_dict[key]

  return response_columns

def incident_duration(ID):

  return_dict = {'days_to_contain': 0, 'days_to_control': 0, 'days_to_out': 0}

  start = dt.strptime(dump_incident.query('IncidentID == @ID')['IncidentDate'].values[0], '%Y-%m-%d %H:%M:%S')
  
  try:
    end = dt.strptime(dump_fireinfo.query('IncidentID == @ID')['OutDate'].values[0], '%Y-%m-%d %H:%M:%S')
    to_out_result = end - start
  except:
    to_out_result = -1

  try:
    end = dt.strptime(dump_fireinfo.query('IncidentID == @ID')['ContainDate'].values[0], '%Y-%m-%d %H:%M:%S')
    to_contain_result = end - start
  except:
    to_contain_result = -1

  try:
    end = dt.strptime(dump_fireinfo.query('IncidentID == @ID')['ControlDate'].values[0], '%Y-%m-%d %H:%M:%S')
    to_control_result = end - start
  except:
    to_control_result = -1

  if to_contain_result == -1:
    return_dict['days_to_contain'] = 'NOT_CONTAINED'
  else:
    return_dict['days_to_contain'] = to_contain_result.days + round(to_contain_result.seconds/3600/24, 3)

  if to_control_result == -1:
    return_dict['days_to_control'] = 'NOT_CONTROLLED'
  else:
    return_dict['days_to_control'] = to_control_result.days + round(to_control_result.seconds/3600/24, 3)

  if to_out_result == -1:
    return_dict['days_to_out'] = 'NOT_OUT'
  else:
    return_dict['days_to_out'] = to_out_result.days + round(to_out_result.seconds/3600/24, 3)

  return return_dict

def compile_incident_meta(ID):
  #incident = get_incident_df(ID)

  #return
  #log_len = len(incident)
  drop_columns = ['ICID', 'IncidentID', 'ICEffective', 'DispatcherID', 'ICComment']
  
  #incident_ic_data  = dump_incident_ic.query('IncidentID == @ID').sort_values(by='ICDate')
  #incident_ic_data = incident_ic_data.drop(columns=drop_columns).reset_index(drop=True).query('IsTrainee == False').drop_duplicates(subset='ICName')
  #ic_list = incident_ic_data['ICName'].drop_
  
  
  drop_columns = list(dump_fireinfo.isna().sum().sort_values()[17:].index)
  drop_columns += ['IAcres', 'Elevation', 'IComplexity', 'InitialStrategyID', 'LastUpdate', 'SizeClass', 'IJurisdiction']
  #return
  fireinfo_data = dump_fireinfo.query('IncidentID == @ID').drop(drop_columns,axis=1).reset_index(drop=True)

  #return
  duration = incident_duration(ID)

  #return
  response = count_response(ID)

  #return


  #return
  try:
    jurisdiction = dump_fireinfo.query('IncidentID == @ID')['SubUnitCode'].values[0]
  except:
    jurisdiction = 'other'
    
  if jurisdiction == '3-NC':
    jurisdiction = 'DNR-NC'
  if jurisdiction == '4-AR':
    jurisdiction = 'DNR-AR'
  if jurisdiction == '1-HI':
    jurisdiction = 'DNR-HI'
  if jurisdiction == 'DNR-SO':
    jurisdiction = 'SO'
  if jurisdiction == '2-SO':
    jurisdiction = 'DNR-SO'
  if jurisdiction == 'D4REP':
    jurisdiction = 'USFS-D4-REP'
  if jurisdiction == 'D12-3':
    jurisdiction = 'USFS-D12-3R'
  if jurisdiction == 'D9-TON':
    jurisdiction = 'USFS-D9-TON'
  if jurisdiction == 'D3-NP':
    jurisdiction = 'USFS-D3-NP'
  if jurisdiction == 'D5-SL':
    jurisdiction = 'USFS-D5-SL'


  #return
  try:
    size = dump_fireinfo.query('IncidentID == @ID')['Acres'].values[0]
  except:
    size = 0

  start = dt.strptime(dump_incident.query('IncidentID == @ID')['IncidentDate'].values[0], '%Y-%m-%d %H:%M:%S')

  try:
    contain_date = dt.strptime(fireinfo_data['ContainDate'].values[0], '%Y-%m-%d %H:%M:%S')
  except:
    contain_date = None
  
  try:
    control_date = dt.strptime(fireinfo_data['ControlDate'].values[0], '%Y-%m-%d %H:%M:%S')
  except:
    control_date = None

  try:
    out_date = dt.strptime(fireinfo_data['OutDate'].values[0], '%Y-%m-%d %H:%M:%S')
  except:
    out_date = None

  incident_meta = dump_incident.query('IncidentID == @ID')
  incident_name = incident_meta['IncidentName'].values[0]
  incident_num = incident_meta['IncidentNum'].values[0]
  incident_year = dt.strptime(incident_meta['IncidentDate'].values[0], '%Y-%m-%d %H:%M:%S').year
  sub_type = int(incident_meta['IncidentSubTypeID'].values[0])
  incident_type = dump_incident_subtype.query('IncidentSubTypeID == @sub_type')['Describe'].values[0].replace("Statistical", '').strip()
  try:
    township = int(incident_meta['ActualTownship'].values[0])
    range = int(incident_meta['ActualRange'].values[0])
    section = int(incident_meta['ActualSection'].values[0])
  except:
    try:
      township = int(incident_meta['Township'].values[0])
      range = int(incident_meta['Range'].values[0])
      section = int(incident_meta['Section'].values[0])
    except:
      township = '?'
      range = '?'
      seciton = '?'

  #icf = dump_incident_center_fires.query('IndicentID == @ID')
  #icf = centerfireID(icf['CenterFireTabID'], icf['CenterFireCatID'])

  try:
    general_cause = dump_incident_center_fires.query('IncidentID == @ID').query('CenterFireTabID == 2 and CenterFireCatID == 1')['Describe'].values[0]
  except:
    general_cause = None

  try:
    specific_cause = dump_incident_center_fires.query('IncidentID == @ID').query('CenterFireTabID == 2 and CenterFireCatID == 2')['Describe'].values[0]
  except:
    specific_cause = None

  try:
    specific_cause_detail = dump_incident_center_fires.query('IncidentID == @ID').query('CenterFireTabID == 2 and CenterFireCatID == 3')['Describe'].values[0]
  except:
    specific_cause_detail = None

  zone = {135: 'UPPER BASIN',
        136: 'FOOTHILLS',
        63: 'COLUMBIA',
        137: 'SHERMAN',
        64: 'HIGHLANDS',
        65: 'OKANOGAN'}
  
  try:
    ret_zone = zone[int(dump_incident.query('IncidentID == @ID')['DispatcherID'].values[0])]
  except:
    ret_zone = 'OTHER'

  return {'global_id': ID,
          'incident_name': incident_name,
          'incident_number': incident_num,
          'incident_year': incident_year,
          'incident_type': incident_type,
          'jurisdiction': jurisdiction,
          'zone': ret_zone,
          'acres_final': size,
          'general_cause': general_cause, 
          'specific_cause': specific_cause,
          'specific_cause_detail': specific_cause_detail,
          'days_to_contain': duration['days_to_contain'],
          'days_to_control': duration['days_to_control'],
          'days_to_out': duration['days_to_out'],
          'start_date': str(start),
          'contain_date': str(contain_date),
          'control_date': str(control_date),
          'out_date': str(out_date),
          'township': township,
          'range': range,
          'section': section,
          'Airtanker': response['Airtanker'], 
          'Fixed Wing': response['Fixed Wing'], 
          'Helicopter': response['Helicopter'], 
          'Handcrew': response['Handcrew'],
          'Module': response['Module'], 
          'Overhead': response['Overhead'], 
          'Dozer': response['Dozer'], 
          'Engine': response['Engine'], 
          'Boat': response['Boat'], 
          'Utility Vehicle': response['Utility Vehicle'],
          'Tractor Plow': response['Tractor Plow'], 
          'Watertender': response['Watertender']
          }


In [ ]:
wildfire_compiled = pd.read_csv('drive/My Drive/NEWICC/wildfire_compiled.csv', index_col=0)

In [ ]:
#all actual fires in database
valid_subtypes = [21, 29, 39, 40, 148, 151, 97, 80]

for i in range(len(dump_incident_subtype)):
  row = dump_incident_subtype.iloc[i]
  if 'DNR Statistical' in row['Describe']:
    valid_subtypes.append(row['IncidentSubTypeID'])

wildfire_incident = dump_incident.query("IncidentTypeID == 2 and IncidentSubTypeID in @valid_subtypes and IncidentYear == 2022")
wildfire_incident = wildfire_incident.reset_index(drop=True)

list_of_dataframes = []

bad_entries = []

for i in range(len(wildfire_incident)):
  #print(i/length)
  id = wildfire_incident.iloc[i]['IncidentID']
  row = wildfire_incident.iloc[i]
  incident_id = row['IncidentID']
  dataframe = pd.DataFrame(compile_incident_meta(incident_id), index=[0])
  list_of_dataframes.append(dataframe)


wildfire_compiled = pd.concat(list_of_dataframes, axis=0).reset_index(drop=True)
wildfire_compiled = wildfire_compiled.drop(columns=['global_id', 'incident_year'])
wildfire_compiled.to_csv('wildfire_compiled.csv')
!cp wildfire_compiled.csv "drive/My Drive/NEWICC"

len(bad_entries)

0

In [ ]:
wildfire_compiled.to_excel('NEWICC_2022_Statistical_Fires.xlsx', sheet_name='2022 Statistical Fires')
!cp NEWICC_2022_Statistical_Fires.xlsx "drive/My Drive/NEWICC"